In [ ]:
datapath = "/Users/joregan/Playing/hsi/"

In [ ]:
from pathlib import Path

DATAPATH = Path(datapath)

In [1]:
OBJECT_HEADER = "TimeCode,ObjectName,ObjectPosX,ObjectPosY,ObjectPosZ"
TIMECODE_HEADER = "Frame,Time (Seconds),TimeCode"

In [7]:
# SAMPLE_OBJECT_HEADER = "Format Version,1.23,Take Name,hsi_7_0719_209_003,Take Notes,,Capture Frame Rate,200.000000,Export Frame Rate,120.000000,Capture Start Time,2024-07-19 03.14.02.737 PM,Capture Start Frame,367318,Total Frames in Take,90242,Total Exported Frames,54146,Rotation Type,Quaternion,Length Units,Millimeters,Coordinate Space,Global"

def read_timecode(filename):
    header_data = {}
    frame_data = []
    header_line_seen = False
    with open(filename) as inf:
        for line in inf.readlines():
            line = line.strip()
            if line.startswith("Format Version,"):
                header_data = {k[0]: k[1] for k in zip(line.split(",")[0::2], line.split(",")[1::2])}
            if not header_line_seen:
                if line.strip() == TIMECODE_HEADER:
                    header_line_seen = True
                else:
                    continue
            else:
                parts = line.split(",")
                frame_data.append({
                    "frame": parts[0],
                    "abs_time": parts[1],
                    "timecode": parts[2]
                })
    return header_data, frame_data

In [10]:
def read_ojects(filename):
    header_seen = False
    object_codes = []
    with open(filename) as inf:
        for line in inf.readlines():
            line = line.strip()
            if not header_seen:
                if line == OBJECT_HEADER:
                    header_seen = True
                else:
                    continue
            else:
                parts = line.split(",")
                timeparts = parts[0].split(":")
                assert timeparts[3][0] == "0", f"in {filename}: non-zero padded time ({line})"
                timeparts[3] = timeparts[3][1:]
                time = ":".join(timeparts)
                object_codes.append({
                    "timecode": time,
                    "name": parts[1],
                    "x": parts[2],
                    "y": parts[3],
                    "z": parts[4]
                })
    return object_codes

In [20]:
test = read_ojects("/Users/joregan/Playing/hsi/objects/hsi_7_0719_209_002_objects.csv")

In [17]:
import json

def id_to_name(filename):
    fwd = {}
    bck = {}
    with open(filename) as inf:
        data = json.load(inf)
    for child in data["origin"]["children"]:
        bck[child["name"]] = child["unique_id"]
        fwd[child["unique_id"]] = child["name"]
    return fwd, bck

In [ ]:
def unique_object_times(data):
    output = []
    

In [32]:
def object_window(data, span=5):
    allcounts = []
    def prcnt():
        out = {}
        for count in counts:
            out[count] = counts[count] / 5
        return out

    for i in range(len(data) - (span - 1)):
        window = data[i:i+5]
        counts = {}
        timecodes = []
        for item in window:
            timecodes.append(item["timecode"])
            if not item["name"] in counts:
                counts[item["name"]] = 1
            else:
                counts[item["name"]] += 1
    
        countspct = prcnt()
        stable = (len(counts) == 1)
        allcounts.append({
            "stable": stable,
            "items": countspct,
            "timecodes": timecodes
        })
    return allcounts

In [33]:
wd = object_window(test)

In [34]:
wd[-1]

{'stable': True,
 'items': {'FloorLamp_4062327b': 1.0},
 'timecodes': ['18:42:45:00.07',
  '18:42:45:01.04',
  '18:42:45:02.02',
  '18:42:45:02.07',
  '18:42:45:04.00']}